### **Import Packages**

In [ ]:
# Libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch import optim, save
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import Sampler
from torch.autograd import Variable

from PIL import Image

#### Parameters for the model

In [ ]:
labels = pd.read_csv('../input/histopathologic-cancer-detection/train_labels.csv')
test = pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')

train_path = '../input/histopathologic-cancer-detection/train/'
test_path = '../input/histopathologic-cancer-detection/test/'

In [ ]:
labels.shape, test.shape

In [ ]:
print("This is number of files in the train: ", len(os.listdir(train_path)))
print("This is number of files in the test: ", len(os.listdir(test_path)))

In [ ]:
# Hyper parameters
num_epochs = 5
num_classes = 2
batch_size = 128
learning_rate = 0.002

# Device configuration
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#### Preview Images

In [ ]:
fig = plt.figure(figsize=(25, 4))
# display 20 images
train_imgs = os.listdir("../input/histopathologic-cancer-detection/train")
for idx, img in enumerate(np.random.choice(train_imgs, 20)):
    ax = fig.add_subplot(2, 20//2, idx+1, xticks=[], yticks=[])
    im = Image.open("../input/histopathologic-cancer-detection/train/" + img)
    plt.imshow(im)
    lab = labels.loc[labels['id'] == img.split('.')[0], 'label'].values[0]
    ax.set_title(f'Label: {lab}')

In [ ]:
example_image = Image.open("../input/histopathologic-cancer-detection/train/" + labels['id'][0] + ".tif")
example_image_numpy = np.array(example_image.getdata())

#Finding the dimensions of the image
print(example_image_numpy.shape)

In [ ]:
#Image dimension determination
print(np.sqrt(example_image_numpy.shape[0]))

In [ ]:
labels.head()

In [ ]:
test.head()

#### Label Percentages

In [ ]:
labels.label.value_counts() / len(labels.label)

In [ ]:
class HS_Dataset(Dataset):
    
    def __init__(self, csv_file, root_dir, transform = None):
        self.df = pd.read_csv(csv_file)
        self.dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, i):
        """This function should return the ith example from the training set.
        The example should be returned in the form of a dictionary: 
        {'image': image_data, 'label': label_data}"""
        
        file = labels['id'][i]
        
        label = np.array(labels['label'][i])
        if label == 0:
            label == 0.0
        else:
            label == 1.0
            
        """Reshape needed to make the output of shape [1]"""
        label = label.reshape((1))
                
        image = Image.open("../input/histopathologic-cancer-detection/train/" + file + ".tif")
        image = np.array(image.getdata()).reshape(96, 96, 3)
        
        sample = {'image': image, 'label': label}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
        
        
class ToTensor(object):
    
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        """This transposition is very important as PyTorch take in the image data in the current shape:
        Number of Channels, Height, Width; So the third axis(channels) in the original image has to 
        be made the first axis."""
        image = image.transpose(2, 0, 1)        
        image = torch.from_numpy(image)
        image = image.type(torch.FloatTensor)
        
        label = torch.from_numpy(label)
        label = label.type(torch.FloatTensor)
        """The optimizer takes in FloatTensor type data. Hence the data has to be converted from any other format
        to FloatTensor type"""
        return {'image': image, 'label': label}

In [ ]:
from torch.utils.data import random_split

hs_dataset = HS_Dataset("../input/histopathologic-cancer-detection/train_labels.csv", "../input/histopathologic-cancer-detection/train/", transform = transforms.Compose([ToTensor()]))

train_size = int(0.995 * len(hs_dataset))
val_size = int((len(hs_dataset) - train_size) / 8)
test_size = int(val_size * 7 / 8)
test_size += len(hs_dataset) - train_size - val_size - test_size

print("train size: ", train_size)
print("val size: ", val_size)
print("test size: ", test_size)

train_data, val_data, test_data = random_split(hs_dataset, [train_size, val_size, test_size])

In [ ]:
#testing the datasete obj

print("training")
for i in range(len(train_data)):
    sample = train_data[i]
    print(i, sample['image'].size(), sample['label'].size())
    if i == 5:
        break

print("validation")
for i in range(len(val_data)):
    sample = val_data[i]
    print(i, sample['image'].size(), sample['label'].size())
    if i == 5:
        break
        
print("testing")
for i in range(len(test_data)):
    sample = test_data[i]
    print(i, sample['image'].size(), sample['label'].size())
    if i == 5:
        break

In [ ]:
# Making the dataloader object

train_loader = DataLoader(dataset = train_data, batch_size = 128, shuffle=True, num_workers=0)
val_loader = DataLoader(dataset= val_data, batch_size = val_size, num_workers=0)
test_loader = DataLoader(dataset= test_data, batch_size = 128, num_workers=0)

print(len(train_loader))
print(len(val_loader))
print(len(test_loader))

In [ ]:
# Making the model architecture

class SkipConvNet(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(SkipConvNet, self).__init__()
        
        self.conv1 = nn.Sequential(nn.Conv2d(num_channels, 8, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(8),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.skip1_1 = nn.Sequential(nn.Conv2d(8, 8, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(8),
                                    nn.ReLU(),
                                    nn.Conv2d(8, 8, kernel_size=3, padding = 1))
        
        self.skip1_2 = nn.Sequential(nn.Conv2d(8, 8, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(8),
                                    nn.ReLU(),
                                    nn.Conv2d(8, 8, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(8),
                                    nn.ReLU(),
                                    nn.Conv2d(8, 8, kernel_size=3, padding = 1))
        
        self.conv2 = nn.Sequential(nn.Conv2d(8, 16, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(16),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.skip2_1 = nn.Sequential(nn.Conv2d(16, 16, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(16),
                                    nn.ReLU(),
                                    nn.Conv2d(16, 16, kernel_size=3, padding = 1))
        
        self.skip2_2 = nn.Sequential(nn.Conv2d(16, 16, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(16),
                                    nn.ReLU(),
                                    nn.Conv2d(16, 16, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(16),
                                    nn.ReLU(),
                                    nn.Conv2d(16, 16, kernel_size=3, padding = 1))
        
        self.conv3 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=3, padding=1),
                                  nn.BatchNorm2d(32),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.skip3_1 = nn.Sequential(nn.Conv2d(32, 32, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(),
                                    nn.Conv2d(32, 32, kernel_size=3, padding = 1))
        
        self.skip3_2 = nn.Sequential(nn.Conv2d(32, 32, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(),
                                    nn.Conv2d(32, 32, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(),
                                    nn.Conv2d(32, 32, kernel_size=3, padding = 1))
        
        self.conv4 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(64),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.skip4_1 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.Conv2d(64, 64, kernel_size=3, padding = 1))
        
        self.skip4_2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.Conv2d(64, 64, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.Conv2d(64, 64, kernel_size=3, padding = 1))
        
        self.conv5 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, padding=1),
                                  nn.BatchNorm2d(128),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.skip5_1 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.Conv2d(128, 128, kernel_size=3, padding = 1))
        
        self.skip5_2 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.Conv2d(128, 128, kernel_size=3, padding = 1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.Conv2d(128, 128, kernel_size=3, padding = 1))
        
         
        self.ff1 = nn.Linear(3 * 3 * 128, 128)
        self.ff2 = nn.Linear(128, 32)
        
        self.output = nn.Linear(32, num_classes)
        
    def forward(self, x):
        out = self.conv1(x)
        out = out + self.skip1_1(out) + self.skip1_2(out)
        out = self.conv2(out)
        out = out + self.skip2_1(out) + self.skip2_2(out)
        out = self.conv3(out)
        out = out + self.skip3_1(out) + self.skip3_2(out)
        out = self.conv4(out)
        out = out + self.skip4_1(out) + self.skip4_2(out)
        out = self.conv5(out)
        out = out + self.skip5_1(out) + self.skip5_2(out)
        out = out.reshape(-1, 3 * 3 * 128)
        out = self.ff1(out)
        out = self.ff2(out)
        out = self.output(out)
        return out

In [ ]:
model = SkipConvNet(3, 1)
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def accuracy_mini_batch(predicted, true, i, acc, tpr, tnr):
    
    predicted = predicted.cpu()
    true = true.cpu()
    
    predicted = (sigmoid(predicted.data.numpy()) > 0.5)
    true = true.data.numpy()
    
    accuracy = np.sum(predicted == true) / true.shape[0]
    true_positive_rate = np.sum((predicted == 1) * (true == 1)) / np.sum(true == 1)
    true_negative_rate = np.sum((predicted == 0) * (true == 0)) / np.sum(true == 0)
    
    acc = acc * (i) / (i + 1)  + accuracy / (i + 1)
    tpr = tpr * (i) / (i + 1)  + true_positive_rate / (i + 1)
    tnr = tnr * (i) / (i + 1) + true_negative_rate / (i + 1)
    
    return acc, tpr, tnr


def accuracy(predicted, true):
    predicted = predicted.cpu()
    true = true.cpu()
    
    predicted = (sigmoid(predicted.data.numpy()) > 0.5)
    true = true.data.numpy()
    
    accuracy = np.sum(predicted == true) / true.shape[0]
    true_positive_rate = np.sum((predicted == 1) * (true == 1)) / np.sum(true == 1)
    true_negative_rate = np.sum((predicted == 0) * (true == 0)) / np.sum(true == 0)

    return accuracy, true_positive_rate, true_negative_rate

In [ ]:
import time
import matplotlib.pyplot as plt

epochs = num_epochs

accuracy_array = []
tpr_array = []
tnr_array = []
loss_array = []

val_loss_array = []
val_acc_array = []
val_tpr_array = []
val_tnr_array = []

## Try without cuda and use GPU 
use_cuda = torch.cuda.is_available()
device = "cuda:0"

for epoch in range(epochs):
    start_time = time.time() 
    
    loss_temp = []
    
    acc, tpr, tnr = 0., 0., 0.
    
    for mini_batch_num, data in enumerate(train_loader):
        images, labels_pic = data['image'], data['label']
        images, labels_pic = images.to(device), labels_pic.to(device)
        
        preds = model(images)
        
        loss = criterion(preds, labels_pic)
        acc, tpr, tnr = accuracy_mini_batch(preds, labels_pic, i, acc, tpr, tnr)
        
        optimizer.zero_grad()
        loss.backward()
        loss_temp.append(loss.item())
        
        optimizer.step()
        if (mini_batch_num) % 4 == 0:
            print ('Epoch {}/{}; Iter {}/{}; Loss: {:.4f}; Acc: {:.3f}; True Pos: {:.3f}; True Neg: {:.3f}'
                   .format(epoch+1, epochs, mini_batch_num + 1, len(train_loader), loss.item(), acc, tpr, tnr), end = "\r", flush = True)
    
    end_time = time.time()
    
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            images, labels_pic = data['image'], data['label']
            images, labels_pic = images.to(device), labels_pic.to(device)
            preds = model(images)
            loss_test = criterion(preds, labels_pic)
            t_acc, t_tpr, t_tnr = accuracy(preds, labels_pic)
    
    val_loss_array.append(loss_test)
    val_acc_array.append(t_acc)
    val_tpr_array.append(t_tpr)
    val_tnr_array.append(t_tnr)
    
    print ('Epoch {}/{}; Loss: {:.4f}; Train Acc: {:.3f}; Train TPR: {:.3f}; Train TNR: {:.3f}; Epoch Time: {} mins; \nTest Loss: {:.4f}; Test Acc: {:.3f}; Test TPR: {:.3f}; Test TNR: {:.3f}\n'
           .format(epoch+1, epochs, loss.item(),acc, tpr, tnr, round((end_time - start_time)/ 60., 2), loss_test, t_acc, t_tpr, t_tnr))
    
    loss_array.append(np.mean(np.array(loss_temp)))
    accuracy_array.append(acc)
    tpr_array.append(tpr)
    tnr_array.append(tnr)

In [ ]:
# Training measures plot

plt.plot(loss_array, color="red")
plt.plot(accuracy_array, color="blue")
plt.plot(tpr_array, color="green")
plt.plot(tnr_array, color="orange")
plt.show()

In [ ]:
# Testing measures plot 

plt.plot(val_loss_array, color="red")
plt.plot(val_acc_array, color="blue")
plt.plot(val_tpr_array, color="green")
plt.plot(val_tnr_array, color="orange")
plt.show()

In [ ]:
print("end of code. Below is test code.")

class MyDataset(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name+'.tif')
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

# RandomCrop, RandomAffine, etc can't be used because they may cause the pixels of tumor tissue
# move away from center
# transforms that changed colours (such as GreyScale) can't be used too
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(30), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, data_dir=train_path, transform=trans_train)
dataset_valid = MyDataset(df_data=val, data_dir=train_path, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

class SimpleCNN(nn.Module):
    def __init__(self):
        # This constructor will initialize the model architecture
        # 512 256 3 3
        # [128, 128, 15, 15] 
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=96, out_channels=128, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(96)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(256)
        self.bn6 = nn.BatchNorm2d(512) 
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(8)
        self.fc = nn.Linear(512 * 1 * 1, 2) 
        self.fc = nn.Sigmoid()
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batchnorm, then activation then pooling layer.
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.pool(F.leaky_relu(self.bn6(self.conv6(x))))
        x = self.avg(x)
        # print(x.shape) # lifehack to find out the correct dimension for the Linear Layer
        x = x.view(-1, 512 * 1 * 1)
        x = self.fc(x)
        return x

model = SimpleCNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
# torch.optim.Adam was also used and showed poorer result during test accuracy

# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
           # print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
           #        .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model on the 22003 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

labels.id = labels.id + '.tif'
test_id = test.id # original ids for submission
test.id = test.id + '.tif'
print(labels.head())
print(test.head())

n = 80000
negative_sample = labels.loc[labels.label == '0', :].sample(n, random_state=1)
positive_sample = labels.loc[labels.label == '1', :].sample(n, random_state=1)
labeled_sample = pd.concat([negative_sample, positive_sample], axis=0).reset_index(drop=True)

train_df, valid_df = train_test_split(labeled_sample, test_size=0.2, random_state=1, stratify=labeled_sample.label)

print(train_df.shape)
print(valid_df.shape)

base_dir = 'images/'
train_dir = 'images/train/'
valid_dir = 'images/valid/'


os.mkdir(base_dir)
os.mkdir(train_dir)
os.mkdir(valid_dir)

os.mkdir(train_dir + 'negative')
os.mkdir(train_dir + 'positive')
os.mkdir(valid_dir + 'negative')
os.mkdir(valid_dir + 'positive')

%%time 

for i in range(len(train_df.id)):
    
    src = train_path + train_df.id.iloc[i]
        
    if train_df.label.iloc[i] == '0':    
        dest = train_dir + 'negative/' + train_df.id.iloc[i]
    else: 
        dest = train_dir + 'positive/' + train_df.id.iloc[i]
        
    shutil.copyfile(src, dest)

print(len(os.listdir(train_dir + 'negative')))
print(len(os.listdir(train_dir + 'positive')))

%%time 

for i in range(len(valid_df.id)):
    
    src = train_path + valid_df.id.iloc[i]
        
    if valid_df.label.iloc[i] == '0':    
        dest = valid_dir + 'negative/' + valid_df.id.iloc[i]
    else: 
        dest = valid_dir + 'positive/' + valid_df.id.iloc[i]
        
    shutil.copyfile(src, dest)

print(len(os.listdir(valid_dir + 'negative')))
print(len(os.listdir(valid_dir + 'positive')))

bs = 64

train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    batch_size = bs,
    shuffle = True,
    class_mode = "binary",
    target_size = (96,96))

valid_generator = train_datagen.flow_from_directory(
    directory = valid_dir,
    batch_size = bs,
    shuffle = True,
    class_mode = "binary",
    target_size = (96,96))

def training_images(seed):
    np.random.seed(seed)
    train_generator.reset()
    imgs, labels = next(train_generator)
        
    plt.figure(figsize=(12,12))
    for i in range(16):
        plt.subplot(4,4,i+1)
        plt.imshow(imgs[i,:,:,:])
        if(labels[i] == 1):
            plt.text(0, -5, 'Positive', color='r')
        else:
            plt.text(0, -5, 'Negative', color='b')
        plt.axis('off')
    plt.show()

training_images(1)

#### Split Train and Test datasets

train, val = train_test_split(labels, stratify=labels.label, test_size=0.1)
len(train), len(val)

#### Simple Custom Generator

class MyDataset(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name+'.tif')
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, data_dir=train_path, transform=trans_train)
dataset_valid = MyDataset(df_data=val, data_dir=train_path, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

im = Image.open(trans_train[0])


#### CNN Model

class SimpleCNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(8)
        self.fc = nn.Linear(512 * 1 * 1, 2) # !!!
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batchnorm, then activation then pooling layer.
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        #print(x.shape) # lifehack to find out the correct dimension for the Linear Layer
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

**Important note:** 

You may notice that in lines with # !!! there is not very clear 512 x 1 x 1. This is the dimension of the picture before the FC layers (H x W x C), then you have to calculate it manually (in Keras, for example, .Flatten () does everything for you). However, there is one life hack — just make print (x.shape) in forward () (commented out line). You will see the size (batch_size, C, H, W) - you need to multiply everything except the first (batch_size), this will be the first dimension of Linear (), and it is in C H W that you need to "expand" x before feeding to Linear ().

model = SimpleCNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)


#### Train the model

# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

#### Accuracy Check

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model on the 22003 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

plt.plot(train_loss, label='Training loss')
plt.plot(test_loss, label='Validation loss')
plt.legend(frameon=False)
plt.show()